# KLASIFIKASI MAKANAN INDONESIA (Gado-gado, Nasi goreng, Soto Ayam, bakso, rendang)

In [8]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

import timm

## KONFIGURASI MODEL DAN DATA

Buat barisan kode "MODEL_NAME = "vit_tiny_patch16_224" menjadi komentar dan buat baris komentar "# MODEL_NAME = "deit_tiny_patch16_224" menjadi barisan kode untuk mengganti model. lakukan hal yang sama untuk path nya.

In [9]:
MODEL_NAME = "vit_tiny_patch16_224"
# MODEL_NAME = "deit_tiny_patch16_224"   # pakai ini untuk model DeiT

CHECKPOINT_PATH = "checkpoints/vit_tiny_best.pth"
# CHECKPOINT_PATH = "checkpoints/deit_tiny_best.pth"


DATA_DIR = "dataset"
TEST_CSV = "uji_coba.csv"
OUTPUT_CSV = "jawaban_klasifikasi.csv"
KUNCI_JAWABAN_CSV = "test.csv"   # file berisi filename + label ground truth

BATCH_SIZE = 32
IMAGE_SIZE = 224
NUM_WORKERS = 0   # aman untuk Windows
DEVICE = torch.device("cpu")  # pakai CPU

CLASSES = ["bakso", "gado_gado", "nasi_goreng", "rendang", "soto_ayam"]
IDX_TO_CLASS = {i: c for i, c in enumerate(CLASSES)}

## DATASET UNTUK TEST (TANPA LABEL)

In [10]:
class FoodTestDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

        # Asumsi kolom wajib bernama 'filename'
        if "filename" not in self.df.columns:
            raise ValueError("test.csv harus memiliki kolom 'filename'")

        self.filenames = self.df["filename"].values

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        img_path = os.path.join(self.img_dir, filename)

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Gambar tidak ditemukan: {img_path}")

        image = Image.open(img_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return image, filename

test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
    ),
])


In [11]:
def create_model(model_name, num_classes):
    model = timm.create_model(
        model_name,
        pretrained=False,          
        num_classes=num_classes
    )
    return model

## FUNGSI HITUNG AKURASI

In [12]:
def hitung_akurasi(pred_csv_path, kunci_csv_path):
    if not os.path.exists(kunci_csv_path):
        print(f"\n[PERINGATAN] File kunci jawaban tidak ditemukan: {kunci_csv_path}")
        print("Akurasi tidak dapat dihitung.")
        return

    df_pred = pd.read_csv(pred_csv_path)
    df_kunci = pd.read_csv(kunci_csv_path)

    if "filename" not in df_kunci.columns or "label" not in df_kunci.columns:
        raise ValueError("kunci_jawaban.csv wajib memiliki kolom 'filename' dan 'label'.")

    if "filename" not in df_pred.columns or "label" not in df_pred.columns:
        raise ValueError("jawaban_klasifikasi.csv wajib memiliki kolom 'filename' dan 'label'.")

    # Rename kolom untuk menghindari bentrok
    df_pred = df_pred.rename(columns={"label": "label_pred"})
    df_kunci = df_kunci.rename(columns={"label": "label_true"})

    # Gabung berdasarkan filename (inner join → hanya yang ada di keduanya)
    df_merged = df_kunci.merge(df_pred, on="filename", how="inner")

    # Buang baris yang labelnya NaN jika ada
    df_merged = df_merged.dropna(subset=["label_true", "label_pred"])

    total = len(df_merged)
    if total == 0:
        print("\n[PERINGATAN] Tidak ada sampel yang bisa dibandingkan antara kunci dan prediksi.")
        return

    benar_mask = df_merged["label_true"] == df_merged["label_pred"]
    benar = benar_mask.sum()
    salah = total - benar
    akurasi = benar / total * 100.0

    print("\n===== HASIL EVALUASI TERHADAP KUNCI JAWABAN =====")
    print(f"Total sampel yang dievaluasi : {total}")
    print(f"Prediksi benar               : {benar}")
    print(f"Prediksi salah               : {salah}")
    print(f"Akurasi                      : {akurasi:.2f}%")

## MAIN INFERENCE

In [13]:
def main():
    print(f"Device: {DEVICE}")
    print(f"Memuat model: {MODEL_NAME}")
    print(f"Checkpoint: {CHECKPOINT_PATH}")

    # Buat model dan load weight
    num_classes = len(CLASSES)
    model = create_model(MODEL_NAME, num_classes)
    state_dict = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    model.load_state_dict(state_dict)
    model.to(DEVICE)
    model.eval()

    # Buat DataLoader test
    test_dataset = FoodTestDataset(TEST_CSV, DATA_DIR, transform=test_transform)
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS
    )

    all_filenames = []
    all_preds = []

    with torch.no_grad():
        pbar = tqdm(test_loader, desc="Inferencing", unit="batch")
        for images, filenames in pbar:
            images = images.to(DEVICE)

            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu().numpy()

            for fname, pred_idx in zip(filenames, preds):
                all_filenames.append(fname)
                all_preds.append(IDX_TO_CLASS[int(pred_idx)])

    # Buat DataFrame hasil
    df_out = pd.DataFrame({
        "filename": all_filenames,
        "label": all_preds
    })

    # Kalau test.csv punya kolom lain, bisa digabung (optional)
    df_test = pd.read_csv(TEST_CSV)
    if "filename" in df_test.columns:
        # merge berdasarkan filename, tapi prioritas label hasil prediksi
        df_merged = df_test.drop(columns=[col for col in df_test.columns if col == "label"], errors="ignore")
        df_merged = df_merged.merge(df_out, on="filename", how="left")
    else:
        df_merged = df_out

    df_merged.to_csv(OUTPUT_CSV, index=False)
    print(f"\nSelesai. Hasil disimpan ke: {OUTPUT_CSV}")
    print(df_merged.head())

    # Hitung akurasi dibanding kunci jawaban
    hitung_akurasi(OUTPUT_CSV, KUNCI_JAWABAN_CSV)


if __name__ == "__main__":
    main()

Device: cpu
Memuat model: vit_tiny_patch16_224
Checkpoint: checkpoints/vit_tiny_best.pth


Inferencing: 100%|██████████| 9/9 [00:12<00:00,  1.36s/batch]


Selesai. Hasil disimpan ke: jawaban_klasifikasi.csv
   filename        label
0  2380.jpg    soto_ayam
1  2372.jpg  nasi_goreng
2  1973.jpg    soto_ayam
3  1171.jpg        bakso
4  1762.jpg    gado_gado

===== HASIL EVALUASI TERHADAP KUNCI JAWABAN =====
Total sampel yang dievaluasi : 279
Prediksi benar               : 275
Prediksi salah               : 4
Akurasi                      : 98.57%
